In [1]:
# Example assuming 'Timestamp' column exists in both dataframes
df_date = pd.concat([df_ind['Timestamp'], df_comp['Date_of_Incident'], df_users['Date']]).drop_duplicates()


# Convert to datetime
df_date = pd.to_datetime(df_date)

# Create DateDimension table
df_date_dimension = pd.DataFrame({
    'Day': df_date.dt.day,
    'Month': df_date.dt.month,
    'Year': df_date.dt.year,
    'Hour_of_Day': df_date.dt.hour
})




NameError: name 'pd' is not defined

In [ ]:
# Combine countries from individual and company dataframes and drop duplicates
df_country = pd.concat([df_ind['Country'], df_comp['Country']]).drop_duplicates()

# Create CountryDimension table
df_country_dimension = pd.DataFrame({
    'Country': df_country
})




In [ ]:
# Extract IP-related columns from df_ind and drop duplicates
df_detailed_location_dimension = df_ind[['Source_IP_Address', 'Destination_IP_Address', 'Valid_Source_IP', 'Valid_Destination_IP']].drop_duplicates()




In [ ]:
# Extract network traffic columns from df_ind and drop duplicates
df_network_traffic_dimension = df_ind[['Source_Port', 'Destination_Port', 'Protocol', 'Packet_Length', 'Packet_Type', 'Traffic_Type', 'Network_Segment']].drop_duplicates()




In [ ]:
# Extract company-related columns from df_comp and drop duplicates
df_company_dimension = df_comp[['Company_Name', 'Industry']].drop_duplicates()



In [ ]:
# Extract device-related columns from df_ind and drop duplicates
df_device_dimension = df_ind[['Browser', 'Operating_System', 'Device_Type']].drop_duplicates()




In [ ]:
df_ind.rename(columns={'IDS/IPS_Alerts': 'IDS_IPS_Alerts'}, inplace=True)
df_ind.rename(columns={'Alerts/Warnings': 'Alerts_Warnings'}, inplace=True)

df_individual_facts = df_ind[['Browser', 'Operating_System', 'Device_Type','Source_Port', 'Destination_Port', 'Protocol', 'Packet_Length', 'Packet_Type', 'Traffic_Type', 'Network_Segment','Source_IP_Address', 'Destination_IP_Address','Country','Attack_Type', 'Payload_Data', 'Malware_Indicators', 'Anomaly_Scores', 'Severity_Level', 'Alerts_Warnings', 'Attack_Signature', 'Action_Taken', 'User_Information', 'Proxy_Information', 'Firewall_Logs', 'IDS_IPS_Alerts', 'Log_Source']]


In [ ]:
df_comp.rename(columns={'Breach_Size_(No._of_Records)': 'Breach_Size_No_of_Records'}, inplace=True)
df_comp.rename(columns={'Mitigation_Time_(Days)': 'Mitigation_Time_Days'}, inplace=True)
df_comp.rename(columns={'Financial_Loss_(USD)': 'Financial_Loss_USD'}, inplace=True)

df_company_facts = df_comp[['Company_Name', 'Industry','Country','Attack_Type', 'Breach_Size_No_of_Records', 'Attack_Vector', 'Mitigation_Time_Days', 'Financial_Loss_USD']]


In [ ]:
df_active_users_facts = df_users[['Active_Online_Users']]


In [ ]:
# Merge DateID into the individual facts
df_individual_facts = pd.merge(df_individual_facts, df_date_dimension, how='left', 
                               left_on=pd.to_datetime(df_ind['Timestamp']).dt.date,
                               right_on=pd.to_datetime(df_date_dimension[['Year', 'Month', 'Day']]).dt.date)

df_individual_facts['DateID'] = df_individual_facts['DateID']

# Merge DateID into the company facts
df_company_facts = pd.merge(df_company_facts, df_date_dimension, how='left', 
                            left_on=pd.to_datetime(df_comp['Date_of_Incident']).dt.date,
                            right_on=pd.to_datetime(df_date_dimension[['Year', 'Month', 'Day']]).dt.date)

df_company_facts['DateID'] = df_company_facts['DateID']

# Merge DateID into the active users facts
df_active_users_facts = pd.merge(df_active_users_facts, df_date_dimension, how='left', 
                                 left_on=pd.to_datetime(df_users['Date']).dt.date,
                                 right_on=pd.to_datetime(df_date_dimension[['Year', 'Month', 'Day']]).dt.date)

df_active_users_facts['DateID'] = df_active_users_facts['DateID']


In [ ]:
print(df_individual_facts.columns)
print(df_country_dimension.columns)


In [ ]:
# Merge CountryID into the individual facts
df_individual_facts = pd.merge(df_individual_facts, df_country_dimension, how='left', 
                               left_on='Country', right_on='Country')

df_individual_facts['CountryID'] = df_individual_facts['CountryID']

# Merge CountryID into the company facts
df_company_facts = pd.merge(df_company_facts, df_country_dimension, how='left', 
                            left_on='Country', right_on='Country')

df_company_facts['CountryID'] = df_company_facts['CountryID']


In [ ]:
# Merge DetailedGeoLocationID into the individual facts
df_individual_facts = pd.merge(df_individual_facts, df_detailed_location_dimension, how='left',
                               left_on=['Source_IP_Address', 'Destination_IP_Address'], 
                               right_on=['Source_IP_Address', 'Destination_IP_Address'])

df_individual_facts['DetailedGeoLocationID'] = df_individual_facts['DetailedGeoLocationID']


In [ ]:
# Merge NetworkTrafficID into the individual facts
df_individual_facts = pd.merge(df_individual_facts, df_network_traffic_dimension, how='left',
                               left_on=['Source_Port', 'Destination_Port', 'Protocol', 'Packet_Length', 'Packet_Type', 'Traffic_Type', 'Network_Segment'],
                               right_on=['Source_Port', 'Destination_Port', 'Protocol', 'Packet_Length', 'Packet_Type', 'Traffic_Type', 'Network_Segment'])

df_individual_facts['NetworkTrafficID'] = df_individual_facts['NetworkTrafficID']


In [ ]:
# Merge DeviceID into the individual facts
df_individual_facts = pd.merge(df_individual_facts, df_device_dimension, how='left', 
                               left_on=['Browser', 'Operating_System', 'Device_Type'], 
                               right_on=['Browser', 'Operating_System', 'Device_Type'])

df_individual_facts['DeviceID'] = df_individual_facts['DeviceID']


In [ ]:
# Merge CompanyID into the company facts
df_company_facts = pd.merge(df_company_facts, df_company_dimension, how='left', 
                            left_on=['Company_Name', 'Industry'], 
                            right_on=['Company_Name', 'Industry'])

df_company_facts['CompanyID'] = df_company_facts['CompanyID']


In [ ]:
df_individual_facts.drop(columns=['Browser', 'Operating_System', 'Device_Type','Source_Port', 'Destination_Port', 'Protocol', 'Packet_Length', 'Packet_Type', 'Traffic_Type', 'Network_Segment','Source_IP_Address', 'Destination_IP_Address','Country'], inplace=True)
df_company_facts.drop(columns=['Company_Name', 'Industry','Country'], inplace=True)

In [ ]:
# Define your database connection parameters
db_user = 'root'  # Replace with your MySQL username
db_password = 'root'  # Replace with your MySQL password
db_host = 'localhost'  # Or your database server hostname
db_name = 'cyber_attacks_warehouse'  # Replace with your database name

# Create the connection string
connection_string = f'mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}'

# Create a connection to the database
engine = create_engine(connection_string)
# Load each dataframe into the respective SQL tables
df_date_dimension.to_sql('datedimension', engine, if_exists='append', index=False)
df_country_dimension.to_sql('countrydimension', engine, if_exists='append', index=False)
df_detailed_location_dimension.to_sql('detailedlocationdimension', engine, if_exists='append', index=False)
df_network_traffic_dimension.to_sql('networktrafficdimension', engine, if_exists='append', index=False)
df_company_dimension.to_sql('companydimension', engine, if_exists='append', index=False)
df_device_dimension.to_sql('devicedimension', engine, if_exists='append', index=False)


df_individual_facts.to_sql('individualcyberattacksfact', engine, if_exists='append', index=False)
df_company_facts.to_sql('companiescyberattacksfact', engine, if_exists='append', index=False)
df_active_users_facts.to_sql('activeonlineusersfact', engine, if_exists='append', index=False)

